In [1]:
import numpy as np


def concatenate(X):
    return np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)


def error(X, y, model):
    return model.predict_proba(X)[:, 1] - y


def prediction(X, model):
    return model.predict_proba(X)[:, 1]


def gradient_(X, y, model, C):
    F = concatenate(X)
    #return F * error(X, y, model)[:, None] + C * w / X.shape[0]
    return F * error(X, y, model)[:, None]


def hessian_(X, model, C):
    probs = prediction(X, model)
    F = concatenate(X)
    H = F.T @ np.diag(probs * (1 - probs)) @ F / F.shape[0] + C * np.eye(
        F.shape[1]) / X.shape[0]
    return H


def inverse_hessian(H):
    return np.linalg.inv(H)

In [2]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.25:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
    
    X_r = X["train"][top_k_index]
    y_r = y["train"][top_k_index]
    X_l = np.delete(X["train"], top_k_index, axis=0)
    y_l = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    #print("prediction", prediction)

    return X_l, y_l, prediction, X_r, y_r



In [3]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction, x_r, y_r = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=0.1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    return new

In [4]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    for k in range(1, np.sum(y["train"])):
        X_l, y_l, pred_change_if, X_r, y_r = Remove(k, delta_pred, test_idx)
        #_, flip,_ = new_train(k, test_idx, delta_pred)
        
        if old > 0.25 and pred_change_if+old <0.25:
            return k
        elif old < 0.25 and pred_change_if+old > 0.25:
            return k
        
    return None

In [5]:
def recursive_NT(test_idx, old, X_0, y_0, model, l2, I=100, D=1):
    eps = 1 / X["train"].shape[0]
    X_l = X_0
    y_l = y_0
    X_r = X_0
    y_r = y_0
    # Compute IP
    F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)
    new_hessain = hessian_(X_l, model, l2)
    new_inv = inverse_hessian(new_hessain)
    new_grad_train = gradient_(X_r, y_r, model, l2)
    delta_k = -eps * new_inv @ new_grad_train.T
    grad_f = F_dev[test_idx] * (old * (1 - old))
    delta_pred = grad_f @ delta_k

    K_new = y_0.shape[0]
    predicted_change_new = None
    ite = 0
    diff = K_new - 0
    removed_order = []
    while (ite < I and diff > D and K_new != 1):
        ite += 1

        if old > 0.25:
            sorted_index = np.flip(delta_pred.argsort())
        else:
            sorted_index = delta_pred.argsort()

        for k in range(1, y_r.shape[0]):
            top_k_index = sorted_index[:k]
            predicted_change = -np.sum(delta_pred[top_k_index])
            """
            print("K", k)
            print("predicted_change", predicted_change)
            print("old", np.round(old))
            print("K", k)
            print("removed shape", X_r.shape)
            print("left shape", X_l.shape)
            print("delta_pred shape", delta_pred.shape)
            print("predicted_change", predicted_change)
            print("old", np.round(old), "new", np.round(old + predicted_change))
            print()
            """
            if (old>0.25) != ((old + predicted_change)>0.25):
                # print("K", k)

                diff = K_new - k
                K_new = k
                predicted_change_new = predicted_change

                index_whole = []
                for idx_r in top_k_index:
                    point = X_r[idx_r]
                    idx_0 = X_dist[str(point.tolist())]
                    index_whole.append(idx_0)

                X_r = X_r[top_k_index]
                y_r = y_r[top_k_index]

                X_l = np.delete(X_0, index_whole, axis=0)
                y_l = np.delete(y_0, index_whole, axis=0)
                # print("removed shape", X_r.shape)
                # print("left shape", X_l.shape)

                # new hessian for the left points
                new_hessain = hessian_(X_l, model, C=l2)
                new_inv = inverse_hessian(new_hessain)
                # new gradient for the removed points
                new_grad_train = gradient_(X_r, y_r, model, C=l2)

                delta_k = -eps * new_inv @ new_grad_train.T
                grad_f = F_dev[test_idx] * (old * (1 - old))
                delta_pred = grad_f @ delta_k
                break

            if k == y_r.shape[0] - 1:
                if K_new == y_0.shape[0] and diff == y_0.shape[0]:
                    return None, predicted_change_new, ite, None, []
                
                return K_new, predicted_change_new, ite, diff, index_whole

    return K_new, predicted_change_new, ite, diff, index_whole

In [6]:
import numpy as np
import warnings
X = {}
y = {}
X["train"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_speech/X_train.npy")
y["train"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_speech/y_train.npy")
X["dev"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_speech/X_dev.npy")
y["dev"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_speech/y_dev.npy")
X["train"].shape, X["dev"].shape

((9632, 300), (1071, 300))

In [7]:
from sklearn.linear_model import LogisticRegression
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

0.8860877684407096

In [8]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inv_H = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * inv_H @ gradient_train.T
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k



In [9]:
import warnings
warnings.filterwarnings("ignore")
X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i
    
NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change")
    print(K_nt, pred_change_nt)
    print("old", old, "new", new_nt)
    print("order", len(order))
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_speech_LR_I100_D1.npy", NT_app_k)
np.save("new_predictions_speech_LR_I100_D1.npy", new_predictions)
np.save("iterations_speech_LR_I100_D1.npy", iterations)
np.save("diffs_speech_LR_I100_D1.npy", diffs)
np.save("order_speech_LR_I100_D1.npy", order_lists)

test_idx 0
K_nt, pred_change
1133 0.19740884554400695
old 0.05260322991434298 new 0.8115478173821786
order 1133

test_idx 1
K_nt, pred_change
22 -0.20002360854301068
old 0.44794800858339784 new 0.26279886283214
order 22

test_idx 2
K_nt, pred_change
None None
old 0.001018952357230555 new None
order 0

test_idx 3
K_nt, pred_change
None None
old 0.07037846178663008 new None
order 0

test_idx 4
K_nt, pred_change
67 -0.20767392320786826
old 0.4554837611675701 new 0.2588119057725591
order 67

test_idx 5
K_nt, pred_change
13 0.04438784653670424
old 0.20574631931889653 new 0.2535418570720089
order 13

test_idx 6
K_nt, pred_change
3 -0.014624682131050164
old 0.2627314255494395 new 0.2478696576025324
order 3

test_idx 7
K_nt, pred_change
333 0.14287123782263164
old 0.10732156290188119 new 0.3454774097815909
order 333

test_idx 8
K_nt, pred_change
164 0.09219273012426077
old 0.1580026011539243 new 0.2729695544266558
order 164

test_idx 9
K_nt, pred_change
None None
old 0.042406765991967764 new N

K_nt, pred_change
None None
old 0.028539701222864828 new None
order 0

test_idx 81
K_nt, pred_change
None None
old 0.012045606403712345 new None
order 0

test_idx 82
K_nt, pred_change
29 -0.08493532721924102
old 0.3346187285027504 new 0.2544557800029519
order 29

test_idx 83
K_nt, pred_change
331 0.11473123710542579
old 0.13528519301929778 new 0.30462734748162235
order 331

test_idx 84
K_nt, pred_change
294 0.1424067487471766
old 0.10777651179527545 new 0.34413181028532003
order 294

test_idx 85
K_nt, pred_change
None None
old 0.036681829503676364 new None
order 0

test_idx 86
K_nt, pred_change
36 0.06934119951330407
old 0.18135149450755883 new 0.2615786880665723
order 36

test_idx 87
K_nt, pred_change
33 -0.09536814505494232
old 0.3433986091063239 new 0.2535130170697621
order 33

test_idx 88
K_nt, pred_change
10 0.05510192532705828
old 0.1971396359134359 new 0.2577327905603279
order 10

test_idx 89
K_nt, pred_change
160 0.13526736533215586
old 0.11506250705456897 new 0.333288607293725

K_nt, pred_change
888 0.14592707448501727
old 0.10412170557541417 new 0.39681371997018555
order 888

test_idx 162
K_nt, pred_change
None None
old 0.014488277979779975 new None
order 0

test_idx 163
K_nt, pred_change
None None
old 0.001018952357230555 new None
order 0

test_idx 164
K_nt, pred_change
None None
old 0.01001188717734213 new None
order 0

test_idx 165
K_nt, pred_change
8 0.02081299224896948
old 0.23061725466069746 new 0.2517755780894403
order 8

test_idx 166
K_nt, pred_change
143 0.105041530412862
old 0.1451179565419395 new 0.2878693122562022
order 143

test_idx 167
K_nt, pred_change
37 0.055553089861047164
old 0.1952577250372355 new 0.2576813881999538
order 37

test_idx 168
K_nt, pred_change
62 -0.1276016179201407
old 0.37707461405758946 new 0.2564138484140341
order 62

test_idx 169
K_nt, pred_change
41 -0.23342316799394414
old 0.48121633269871256 new 0.2645180266782797
order 41

test_idx 170
K_nt, pred_change
19 -0.03807496292727163
old 0.2877161909095407 new 0.25028335787

K_nt, pred_change
None None
old 0.05567050315957601 new None
order 0

test_idx 243
K_nt, pred_change
38 0.05381157465526008
old 0.19688668395550485 new 0.2566187117621666
order 38

test_idx 244
K_nt, pred_change
24 -0.15671517422973036
old 0.4058136252852435 new 0.26005916539830704
order 24

test_idx 245
K_nt, pred_change
248 0.1296411254527089
old 0.12057978879877014 new 0.33192535585826477
order 248

test_idx 246
K_nt, pred_change
2 0.005671807357135348
old 0.24446729127801936 new 0.25083640470880564
order 2

test_idx 247
K_nt, pred_change
305 0.12568015227875112
old 0.12455219787546234 new 0.32417449108448027
order 305

test_idx 248
K_nt, pred_change
171 0.1060595419270431
old 0.14414481519354413 new 0.2947576806784662
order 171

test_idx 249
K_nt, pred_change
None None
old 0.032310677865439334 new None
order 0

test_idx 250
K_nt, pred_change
806 0.1852790917195402
old 0.0647865237889014 new 0.7126739000523808
order 806

test_idx 251
K_nt, pred_change
1276 0.17035521082648342
old 0.

K_nt, pred_change
None None
old 0.0584727704605885 new None
order 0

test_idx 322
K_nt, pred_change
365 0.1623802888266972
old 0.08781742979224744 new 0.44848561624734623
order 365

test_idx 323
K_nt, pred_change
1419 0.16847228534089798
old 0.08155329599924753 new 0.5467775777321823
order 1419

test_idx 324
K_nt, pred_change
None None
old 0.045606982189233675 new None
order 0

test_idx 325
K_nt, pred_change
39 0.07993100876113184
old 0.17117516097000118 new 0.2619891684041397
order 39

test_idx 326
K_nt, pred_change
425 0.13247380208791942
old 0.11762888419828606 new 0.34209667313949976
order 425

test_idx 327
K_nt, pred_change
189 0.09411139467403085
old 0.15591438148907777 new 0.2669320685607222
order 189

test_idx 328
K_nt, pred_change
None None
old 0.015838167615661307 new None
order 0

test_idx 329
K_nt, pred_change
536 0.15489322648815607
old 0.0951129800912105 new 0.42283775810547475
order 536

test_idx 330
K_nt, pred_change
None None
old 0.023715466896448538 new None
order 0



...

K_nt, pred_change
177 -0.36530998009855414
old 0.6142337157877812 new 0.2317030350600668
order 177

test_idx 721
K_nt, pred_change
70 -0.13854653814510987
old 0.38843905997497014 new 0.2594549300427619
order 70

test_idx 722
K_nt, pred_change
1422 0.18409905890199285
old 0.06594665512576724 new 0.6899749150417175
order 1422

test_idx 723
K_nt, pred_change
None None
old 0.043703770731868236 new None
order 0

test_idx 724
K_nt, pred_change
27 0.06353064408326449
old 0.18652693030897832 new 0.2570616112910744
order 27

test_idx 725
K_nt, pred_change
None None
old 0.023201437296476595 new None
order 0

test_idx 726
K_nt, pred_change
977 0.14585701405663734
old 0.1041532834863931 new 0.3648460055776106
order 977

test_idx 727
K_nt, pred_change
1230 0.17625677941379506
old 0.0737826824100445 new 0.5641192252583049
order 1230

test_idx 728
K_nt, pred_change
474 0.1714820706624569
old 0.07860427968829009 new 0.5096437275721977
order 474

test_idx 729
K_nt, pred_change
1787 0.18895316644002458


K_nt, pred_change
None None
old 0.026568142015840096 new None
order 0

test_idx 801
K_nt, pred_change
None None
old 0.06150237279707345 new None
order 0

test_idx 802
K_nt, pred_change
None None
old 0.0511309500656565 new None
order 0

test_idx 803
K_nt, pred_change
30 0.0516840794618907
old 0.19845682477741533 new 0.2539668914104202
order 30

test_idx 804
K_nt, pred_change
265 0.12620737529064174
old 0.12394294880065607 new 0.3230430052323845
order 265

test_idx 805
K_nt, pred_change
1026 0.17398528617906217
old 0.07608616101441276 new 0.5587281459404434
order 1026

test_idx 806
K_nt, pred_change
716 0.13362115492209134
old 0.11642217675659958 new 0.32152786332716765
order 716

test_idx 807
K_nt, pred_change
None None
old 0.03331091011042286 new None
order 0

test_idx 808
K_nt, pred_change
88 0.07061508221343199
old 0.17944656104734055 new 0.2641929674372255
order 88

test_idx 809
K_nt, pred_change
414 0.1354568429333466
old 0.11468035818380061 new 0.3395098484374656
order 414

test_i

K_nt, pred_change
1086 0.17311624399423747
old 0.07693216526318467 new 0.570226144577195
order 1086

test_idx 881
K_nt, pred_change
166 0.11158427189007075
old 0.13849931586363046 new 0.2957623450767235
order 166

test_idx 882
K_nt, pred_change
489 0.15145619951732203
old 0.09867131120512351 new 0.33730283810122946
order 489

test_idx 883
K_nt, pred_change
None None
old 0.00992074502311751 new None
order 0

test_idx 884
K_nt, pred_change
23 0.027069317396390787
old 0.2235290925848052 new 0.25173015510346486
order 23

test_idx 885
K_nt, pred_change
116 0.12766809541555066
old 0.12294574435864605 new 0.2847316997306206
order 116

test_idx 886
K_nt, pred_change
49 0.05439648775071284
old 0.19582566888311873 new 0.2558393709102144
order 49

test_idx 887
K_nt, pred_change
None None
old 0.01211274112063222 new None
order 0

test_idx 888
K_nt, pred_change
46 0.08920106054526807
old 0.16217421441878602 new 0.27053049778043053
order 46

test_idx 889
K_nt, pred_change
None None
old 0.02150640351

K_nt, pred_change
None None
old 0.06462831905557906 new None
order 0

test_idx 961
K_nt, pred_change
1379 0.18467279696646288
old 0.0653816184644433 new 0.6661099510753968
order 1379

test_idx 962
K_nt, pred_change
None None
old 0.01677143301183885 new None
order 0

test_idx 963
K_nt, pred_change
None None
old 0.06338314818762972 new None
order 0

test_idx 964
K_nt, pred_change
463 0.15980321279235937
old 0.09026046272644896 new 0.34964354581965634
order 463

test_idx 965
K_nt, pred_change
146 0.10470353417776151
old 0.1456218333701522 new 0.2879489092711676
order 146

test_idx 966
K_nt, pred_change
77 0.0634728872316787
old 0.18658745810474903 new 0.25917466437739606
order 77

test_idx 967
K_nt, pred_change
53 0.09198695257562986
old 0.15919079717385862 new 0.2733131083990813
order 53

test_idx 968
K_nt, pred_change
914 0.1566462007077572
old 0.0933756526321391 new 0.41165343388623976
order 914

test_idx 969
K_nt, pred_change
1126 0.19700004073687313
old 0.05305546000417131 new 0.7313

K_nt, pred_change
159 0.13937881164127192
old 0.11091758323516894 new 0.34307278174409517
order 159

test_idx 1043
K_nt, pred_change
None None
old 0.028637213563474064 new None
order 0

test_idx 1044
K_nt, pred_change
615 0.16666402907559585
old 0.08334935625106381 new 0.49302525511879647
order 615

test_idx 1045
K_nt, pred_change
None None
old 0.017590002935330515 new None
order 0

test_idx 1046
K_nt, pred_change
11 -0.039080633573489995
old 0.2890752937269725 new 0.2507448287703143
order 11

test_idx 1047
K_nt, pred_change
33 0.043930340377233146
old 0.20633099116539272 new 0.2539792067971893
order 33

test_idx 1048
K_nt, pred_change
65 -0.28065825836102154
old 0.5280439251237484 new 0.2571010203709302
order 65

test_idx 1049
K_nt, pred_change
None None
old 0.020309477619397123 new None
order 0

test_idx 1050
K_nt, pred_change
137 0.10119283622697936
old 0.1489917980084968 new 0.279953866181475
order 137

test_idx 1051
K_nt, pred_change
75 0.04455214208227688
old 0.20554855558689447 

In [10]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF+Newton/Server_all_experiments/recursive_all/speech'

# Bert lin

In [11]:
X = {}
y = {}
X["train"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_speech/train_feature_save.npy")
y["train"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_speech/train_label_save.npy").squeeze()
X["dev"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_speech/test_feature_save.npy")
y["dev"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_speech/test_label_save.npy").squeeze()


# In[14]:


X["train"].shape, X["dev"].shape, y["train"].shape, y["dev"].shape

((9632, 768), (1071, 768), (9632,), (1071,))

In [12]:
# In[15]:


l2 = 10
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]

#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_H = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_H @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape


# In[18]:


X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i

NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()
    print("old", old)

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change_nt, ite, diff")
    print(K_nt, pred_change_nt, ite, diff)
    print("new", new_nt)
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_speech_bertT_I100_D1.npy", NT_app_k)
np.save("new_predictions_speech_bertT_I100_D1.npy", new_predictions)
np.save("iterations_speech_bertT_I100_D1.npy", iterations)
np.save("diffs_speech_bertT_I100_D1.npy", diffs)
np.save("order_speech_bertT_I100_D1.npy", order_lists)

test_idx 0
old 0.128703605960157
K_nt, pred_change_nt, ite, diff
58 0.12274292053992462 3 6
new 0.30884246697180734

test_idx 1
old 0.010235158870847902
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 2
old 0.015657428242181137
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 3
old 0.07829030304139381
K_nt, pred_change_nt, ite, diff
236 0.17195546004933307 3 65
new 0.5243631514553101

test_idx 4
old 0.07740144357028349
K_nt, pred_change_nt, ite, diff
295 0.17293703095317858 3 126
new 0.5257677544428094

test_idx 5
old 0.10602196408258405
K_nt, pred_change_nt, ite, diff
211 0.14431136800405814 3 46
new 0.37299394151472626

test_idx 6
old 0.20135129231237026
K_nt, pred_change_nt, ite, diff
23 0.04924141937872982 2 1
new 0.2559236737379955

test_idx 7
old 0.03876842169859475
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 8
old 0.2909261775789655
K_nt, pred_change_nt, ite, diff
5 -0.04333016406676994 2 9627
new 0.2484688678846479


K_nt, pred_change_nt, ite, diff
93 -0.2554578600232845 3 13
new 0.26146193338714757

test_idx 77
old 0.01447310468279735
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 78
old 0.028052036837091017
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 79
old 0.018993391914492178
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 80
old 0.14707565328058392
K_nt, pred_change_nt, ite, diff
75 0.10293359686892975 3 8
new 0.2990931487215407

test_idx 81
old 0.04051791504295762
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 82
old 0.16931273820475565
K_nt, pred_change_nt, ite, diff
52 0.08134207333888405 3 4
new 0.27074246312689876

test_idx 83
old 0.03613817941293454
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 84
old 0.0187794984161115
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 85
old 0.02357727093023944
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 8

K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 154
old 0.012139914286179231
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 155
old 0.13348843986161146
K_nt, pred_change_nt, ite, diff
82 0.11683927386094677 3 10
new 0.29632218957959566

test_idx 156
old 0.16012630849817788
K_nt, pred_change_nt, ite, diff
71 0.08993517019015841 3 7
new 0.28107603323213853

test_idx 157
old 0.027780503910573417
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 158
old 0.5694062716398206
K_nt, pred_change_nt, ite, diff
81 -0.32104240190634503 3 15
new 0.24955985347718235

test_idx 159
old 0.028258412493979222
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 160
old 0.06521836614752692
K_nt, pred_change_nt, ite, diff
549 0.1849328436355428 3 412
new 0.6432208208618364

test_idx 161
old 0.41289702157878605
K_nt, pred_change_nt, ite, diff
32 -0.1641483668933063 3 4
new 0.261544976353865

test_idx 162
old 0.005748806601332801
K_nt, 

K_nt, pred_change_nt, ite, diff
1 0.003745395376080999 1 9631
new 0.2548515430460878

test_idx 231
old 0.7648764968501715
K_nt, pred_change_nt, ite, diff
313 -0.5149886289705545 3 183
new 0.09744431493705691

test_idx 232
old 0.0728069976972817
K_nt, pred_change_nt, ite, diff
293 0.17720241631218636 3 98
new 0.5417517826094571

test_idx 233
old 0.08115229219799905
K_nt, pred_change_nt, ite, diff
429 0.16885478910705104 3 216
new 0.4736829444163966

test_idx 234
old 0.11525573969038125
K_nt, pred_change_nt, ite, diff
161 0.13490217186069453 3 39
new 0.3297037482222324

test_idx 235
old 0.012633584724403554
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 236
old 0.2291866068281937
K_nt, pred_change_nt, ite, diff
10 0.021885245579924523 2 9622
new 0.25128971041705667

test_idx 237
old 0.12981998233884054
K_nt, pred_change_nt, ite, diff
116 0.12059736248440792 3 18
new 0.3124588432720889

test_idx 238
old 0.5316319499945634
K_nt, pred_change_nt, ite, diff
65 -0.28339122

K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 305
old 0.019945977904374548
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 306
old 0.056432118791205986
K_nt, pred_change_nt, ite, diff
380 0.19367618734940958 3 136
new 0.7421054485415846

test_idx 307
old 0.04311501846092393
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 308
old 0.10464885840869305
K_nt, pred_change_nt, ite, diff
135 0.1455625703074734 3 32
new 0.36151772439010843

test_idx 309
old 0.058251625742887565
K_nt, pred_change_nt, ite, diff
938 0.19175737956290173 3 661
new 0.7676251108837483

test_idx 310
old 0.15027631137249392
K_nt, pred_change_nt, ite, diff
80 0.10004873629617059 3 13
new 0.28126091130964237

test_idx 311
old 0.12937958627679277
K_nt, pred_change_nt, ite, diff
129 0.12110582549216356 3 19
new 0.31708571929874935

test_idx 312
old 0.027262008790485095
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 313
old 0.05339874654454597

...

K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 751
old 0.09240440361007969
K_nt, pred_change_nt, ite, diff
128 0.15810179294535115 3 25
new 0.42962967577384076

test_idx 752
old 0.0508615114559188
K_nt, pred_change_nt, ite, diff
1149 0.19920601973009205 4 82
new 0.8318810109582642

test_idx 753
old 0.4736002309234426
K_nt, pred_change_nt, ite, diff
68 -0.22538379378128803 3 10
new 0.262049620201254

test_idx 754
old 0.04144549476868969
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 755
old 0.038915739583108434
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 756
old 0.019747794889902186
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 757
old 0.025293327646895555
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 758
old 0.2154992574086674
K_nt, pred_change_nt, ite, diff
24 0.0352958877466418 2 1
new 0.2563783855772188

test_idx 759
old 0.015581497180631239
K_nt, pred_change_nt, ite, diff
N

K_nt, pred_change_nt, ite, diff
164 0.13800945616576704 3 34
new 0.3453029515343595

test_idx 826
old 0.1443878090680525
K_nt, pred_change_nt, ite, diff
98 0.10572851556634598 3 12
new 0.2813189347496727

test_idx 827
old 0.08777421461657461
K_nt, pred_change_nt, ite, diff
307 0.16252997443979006 3 126
new 0.44722627918498586

test_idx 828
old 0.03238836647217911
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 829
old 0.049510825068425544
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 830
old 0.015674240436139613
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 831
old 0.017593610239728403
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 832
old 0.06564003880413345
K_nt, pred_change_nt, ite, diff
407 0.18451229290918186 3 261
new 0.5862383307209524

test_idx 833
old 0.07327704439580905
K_nt, pred_change_nt, ite, diff
314 0.17687035949203866 3 132
new 0.549739017883797

test_idx 834
old 0.009948364008706786
K

K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 901
old 0.11605947138568097
K_nt, pred_change_nt, ite, diff
163 0.13394131758618713 3 29
new 0.3407134895729386

test_idx 902
old 0.06335200494951755
K_nt, pred_change_nt, ite, diff
536 0.18670304772764967 3 401
new 0.6731376707910254

test_idx 903
old 0.06399576325818863
K_nt, pred_change_nt, ite, diff
377 0.1861455922734256 3 206
new 0.6548204394373235

test_idx 904
old 0.3933395553140581
K_nt, pred_change_nt, ite, diff
22 -0.1449275804195691 3 2
new 0.26180193070369995

test_idx 905
old 0.01903586711355233
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 906
old 0.04427707676362778
K_nt, pred_change_nt, ite, diff
1125 0.20573078414970591 4 248
new 0.9138873488176584

test_idx 907
old 0.03680795511201606
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 908
old 0.01871926483016891
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 909
old 0.09615285440077559
K_nt,

K_nt, pred_change_nt, ite, diff
11 0.047206317368953594 2 1
new 0.25485089204980294

test_idx 976
old 0.10600334238081388
K_nt, pred_change_nt, ite, diff
105 0.14462669288247237 3 17
new 0.3714984546066639

test_idx 977
old 0.034861654377720507
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 978
old 0.023547315812219638
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 979
old 0.03607292708850945
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 980
old 0.017462450204250652
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 981
old 0.4324126963929716
K_nt, pred_change_nt, ite, diff
61 -0.18360624126894878 3 7
new 0.25956277876867956

test_idx 982
old 0.20060256165285992
K_nt, pred_change_nt, ite, diff
21 0.051233662998018244 2 1
new 0.2649266894553326

test_idx 983
old 0.04741478068347337
K_nt, pred_change_nt, ite, diff
918 0.20261438230127704 4 173
new 0.8026882362477681

test_idx 984
old 0.00715209937895391
K_nt

K_nt, pred_change_nt, ite, diff
34 0.07614733309801149 3 2
new 0.2698642375314134

test_idx 1051
old 0.46372684783664486
K_nt, pred_change_nt, ite, diff
49 -0.21412641678861985 3 6
new 0.2577544634448162

test_idx 1052
old 0.01593135917168484
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 1053
old 0.15766055514879784
K_nt, pred_change_nt, ite, diff
66 0.09337670112378277 3 5
new 0.27621988756230864

test_idx 1054
old 0.06807970146506546
K_nt, pred_change_nt, ite, diff
456 0.18192991865836727 3 187
new 0.6368027715145198

test_idx 1055
old 0.46775220249297783
K_nt, pred_change_nt, ite, diff
61 -0.2188377805452979 3 7
new 0.2650316685778916

test_idx 1056
old 0.010233218282104706
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 1057
old 0.05832906047709894
K_nt, pred_change_nt, ite, diff
581 0.19173440620518856 3 739
new 0.6859209597273965

test_idx 1058
old 0.19567271928034746
K_nt, pred_change_nt, ite, diff
32 0.054662467300855756 3 2
new 0.25912